In [1]:
import os
import re 
import json
import requests
import datetime
import dateparser
import pandas as pd

In [8]:
with open('metadata/languages.json') as f:
    language_dict = json.load(f)

with open('metadata/pages.json') as f:
    num_pages = json.load(f)

with open('metadata/corrections.json') as f:
    num_corrections = json.load(f)

## Clean Test

In [2]:
df_correction = pd.read_csv('../data/uk_corrections.csv').reset_index(drop=True)
print("Clean - before: {}".format(df_correction.shape[0]))

def pre_clean_data(df):
    df = df[~df['text_del'].str.startswith('|')].copy()
    df = df[~df['text_del'].str.startswith('==')].copy()
    df = df[~df['text_del'].str.startswith('*')].copy()
    df = df[~df['text_del'].str.startswith('!')].copy()

    df = df[~df['text_ins'].str.startswith('|')].copy()
    df = df[~df['text_ins'].str.startswith('==')].copy()
    df = df[~df['text_ins'].str.startswith('*')].copy()
    df = df[~df['text_ins'].str.startswith('!')].copy()

    return df.reset_index(drop=True) 

df_correction = pre_clean_data(df_correction)
print("Clean - after: {}".format(df_correction.shape[0]))
df_correction.tail(3)

Clean - before: 6165
Clean - after: 4734


,delitions,n_del,text_del,text_del_tag,insertions,n_ins,text_ins,text_ins_tag,diff_url,diff_page
4731,[['НКВД']],1,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,[['НКВС']],1,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,https://uk.wikipedia.org/w/index.php?title=%D0...,0
4732,[['зібра но']],1,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,[['зібрано']],1,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,https://uk.wikipedia.org/w/index.php?title=%D0...,2
4733,[[' ']],1,На межі ХІХ - ХХ століть кількість населення С...,На межі ХІХ - ХХ століть кількість населення С...,[],0,На межі ХІХ - ХХ століть кількість населення С...,На межі ХІХ - ХХ століть кількість населення С...,https://uk.wikipedia.org/w/index.php?title=%D0...,2


In [3]:
def clean_text(text):

    # remove square brackets and leave only the last elements of text from brackets
    text = re.sub(r'\[\[(.*?)\]\]', lambda x: x.group(1).rsplit('|', 1)[-1], text)
    text = re.sub(r'\[(.*?)\]', lambda x: ' '.join(x.group(1).split(' ')[1:]), text)

    # remove referral links
    text = re.sub(r'<ref .\S* />', '', text)
    text = re.sub(r'<ref .\S*/>', '', text)
    text = re.sub(r'<ref(.*?)</ref>', '', text)
#     text = re.sub(r'<ref(.*?)>', '', text)
    text = re.sub(r'.*</ref>', '', text)
    text = re.sub(r'<ref(.*?)$', '', text)
    text = re.sub(r'<!--(.*?)-->', '', text)
    
    text = re.sub(r'{{Citation needed}}', '', text)
    text = re.sub(r'{{cit(.*?)$|{{Cit(.*?)$', '', text)
    text = re.sub(r'{{#tag(.*?)}}', '', text)

    text = re.sub('\|en', '', text)
    text = re.sub('<sup></sup>', '', text)
    text = re.sub('\|\|', '|', text)
    text = re.sub('\|\d+=en', '', text)
    text = re.sub('<br.*/>', '', text)
    text = re.sub('<u>.*</u>', '', text)
    text = re.sub('<noinclude>.*</noinclude>', '', text)
    text = re.sub('<code>.*</code>', '', text)

    # remove iw tag and leave only the last elements of text from brackets
    text = re.sub(r'{{iw(.*?)}}', lambda x: x.group(1).rsplit('|', 1)[-1], text)
    # {{нп|Лагоська колонія||Lagos_Colony}}
    text = re.sub(r'{{нп\|(.*?)}}', lambda x: x.group(1).split('|', 1)[0], text)
    # {{МФА2|β̞e.neˈswe.la}}
    text = re.sub(r'{{МФА(.*?)}}', '', text)
    # {{Überarbeiten|grund=Sehr knappe Darstellung der Karriere.}}
    text = re.sub(r'{{Überarbeiten(.*?)}}', '', text)
    # {{formatnum:77000}} -> 77000
    text = re.sub(r'{{formatnum:(.*?)}}', lambda x: x.group(1), text)
    
    text = re.sub(r'{{lang(.*?)}};|{{Lang(.*?)}};', '', text)
    text = re.sub(r'{{lang(.*?)}}|{{Lang(.*?)}}', '', text)
    text = re.sub(r'{{Lang(.*?)}}', '', text)

    # {{s|В. В. Трепавлову}  
    text = re.sub(r'{{s\|(.*?)}}', lambda x: x.group(1).rsplit('|', 1)[-1], text)
    text = re.sub(r'{{Bt-(.*?)}}', lambda x: x.group(1).split('|', 0)[-1], text)

    # {{rp|<small>ст.26</small>}}
    text = re.sub(r'{{rp\|(.*?)}}', '', text)
    # {{Webarchive|url=https://web.archive.org/web/20160930153621/https://www.youtube.com/watch?v=s6NTv3aWeAo&feature=relmfu |date=30 вересня 2016 }})
    text = re.sub(r'{{Webarchive\|(.*?)}}', '', text)
    text = re.sub(r'{{Speciesbox\|(.*?)}}', '', text)
    text = re.sub(r'{{Sfn\|(.*?)}}|{{sfn\|(.*?)}}', '', text)
    text = re.sub(r'{{efn(.*?)}}', '', text)
    text = re.sub(r'{{Ref(.*?)}}', '', text)
    text = re.sub(r'{{R\|Adamy}}', '', text)

    text = re.sub(r'{{цитата\|(.*?)}}', lambda x: x.group(1).rsplit('|', 1)[-1], text)
#     text = re.sub(r'{{start date(.*?)}}', lambda x: x.group(1).rsplit('|', 1)[-2], text)

    text = re.sub(r'<nowiki/>', '', text)
    text = re.sub(r'\'\'\'', '', text)
    
    text = re.sub(r"«''", "«", text)
    text = re.sub(r"»''", "»", text)
    text = re.sub(r"''«", "«", text)
    text = re.sub(r"''»", "»", text)
    text = re.sub(r"''", '"', text)

    text = re.sub(r"»", '"', text)
    text = re.sub(r"«", '"', text)

    # remove &nbsp
    text = re.sub(r'&nbsp;', ' ', text)
    # replace \xa0 to ''
    text = re.sub(r'\xa0', ' ', text)

    text = re.sub(r'\(\)', '', text)
    text = re.sub(' +\.', '.', text)
    text = re.sub(' +\,', '.', text)
    text = re.sub(' +', ' ', text)

    return text.strip()

id__ = 728
line = df_correction.iloc[id__]['text_del']
print(line)
print()
print(clean_text(line))

1724 року три міста, численні селища, а також замок, який до цього не входив до складу жодного з міст, а мав особливий статус урядової резиденції&nbsp;— були об'єднані в місто Кенігсберг.

1724 року три міста, численні селища, а також замок, який до цього не входив до складу жодного з міст, а мав особливий статус урядової резиденції — були об'єднані в місто Кенігсберг.


In [4]:
df_correction['text_del_clean'] = df_correction['text_del'].apply(clean_text)
df_correction['text_ins_clean'] = df_correction['text_ins'].apply(clean_text)
# df_correction['text_del_tag_clean'] = df_correction['text_del_tag'].apply(clean_text)
# df_correction['text_ins_tag_clean'] = df_correction['text_ins_tag'].apply(clean_text)

In [5]:
def post_clean_data(df):
    len_text = 50
    number_correction = 10 
    df = df[df['text_del_clean']!=df['text_ins_clean']].copy()
    df = df[df['text_del_clean'].apply(len)>len_text].copy()
    df = df[df['n_del']<=number_correction].copy()
    df = df[df['n_ins']<=number_correction].copy()
    return df.reset_index(drop=True) 

df_correction_ = post_clean_data(df_correction)
df_correction_.tail(3)

,delitions,n_del,text_del,text_del_tag,insertions,n_ins,text_ins,text_ins_tag,diff_url,diff_page,text_del_clean,text_ins_clean
3635,[['бів']],1,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,[['був']],1,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,https://uk.wikipedia.org/w/index.php?title=%D0...,0,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...
3636,[['НКВД']],1,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,[['НКВС']],1,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,https://uk.wikipedia.org/w/index.php?title=%D0...,0,У жовтні 1939 року о. Йосиф дозволив усім бель...,У жовтні 1939 року о. Йосиф дозволив усім бель...
3637,[['зібра но']],1,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,[['зібрано']],1,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,https://uk.wikipedia.org/w/index.php?title=%D0...,2,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...


## Clean Dataset

In [6]:
%%time
output_list = []
for i in sorted(os.listdir('../data/')):
    if i.split('_')[-1]=='corrections.csv':
        print(i)
        df_correction_temp = pd.read_csv(os.path.join('../data', i))
        if not df_correction_temp.empty:
            print('  Input:', df_correction_temp.shape[0])
            df_correction_temp = pre_clean_data(df_correction_temp)
            print('  Pre Processing:', df_correction_temp.shape[0])

            df_correction_temp['text_del_clean'] = df_correction_temp['text_del'].apply(clean_text)
            df_correction_temp['text_ins_clean'] = df_correction_temp['text_ins'].apply(clean_text)

            df_correction_temp = post_clean_data(df_correction_temp)
            print('  Post Processing:', df_correction_temp.shape[0])

            df_correction_temp['code_lang'] = i.split('_')[0]
            output_list.append(df_correction_temp)

cs_corrections.csv
  Input: 1193
  Pre Processing: 952
  Post Processing: 746
de_corrections.csv
  Input: 4672
  Pre Processing: 3141
  Post Processing: 2386
el_corrections.csv
  Input: 562
  Pre Processing: 412
  Post Processing: 302
en_corrections.csv
  Input: 29454
  Pre Processing: 21171
  Post Processing: 15217
et_corrections.csv
  Input: 138
  Pre Processing: 114
  Post Processing: 90
is_corrections.csv
it_corrections.csv
  Input: 6488
  Pre Processing: 5530
  Post Processing: 3983
lv_corrections.csv
  Input: 75
  Pre Processing: 47
  Post Processing: 33
sl_corrections.csv
  Input: 121
  Pre Processing: 72
  Post Processing: 47
sv_corrections.csv
  Input: 654
  Pre Processing: 510
  Post Processing: 408
uk_corrections.csv
  Input: 6165
  Pre Processing: 4734
  Post Processing: 3638
CPU times: user 20.9 s, sys: 262 ms, total: 21.2 s
Wall time: 21.2 s


In [9]:
df_output = pd.concat(output_list)
df_output = df_output.drop_duplicates(['text_del', 'text_ins'], keep='last')
df_output = df_output.reset_index(drop=True).reset_index()
df_output['language'] = df_output['code_lang'].replace(language_dict)
df_output.tail(3)

,index,delitions,n_del,text_del,text_del_tag,insertions,n_ins,text_ins,text_ins_tag,diff_url,diff_page,text_del_clean,text_ins_clean,code_lang,language
26823,26823,[['бів']],1,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,[['був']],1,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,https://uk.wikipedia.org/w/index.php?title=%D0...,0,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,uk,ukrainian
26824,26824,[['НКВД']],1,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,[['НКВС']],1,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,https://uk.wikipedia.org/w/index.php?title=%D0...,0,У жовтні 1939 року о. Йосиф дозволив усім бель...,У жовтні 1939 року о. Йосиф дозволив усім бель...,uk,ukrainian
26825,26825,[['зібра но']],1,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,[['зібрано']],1,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,https://uk.wikipedia.org/w/index.php?title=%D0...,2,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,uk,ukrainian


## Statistics

In [10]:
df_num_corrections_clean = df_output['language'].value_counts().reset_index()\
        .rename(columns={'index': 'language', 'language': '# edits clean'})
display(df_num_corrections_clean)

with open('metadata/corrections_clean.json', 'w', encoding='utf-8') as f:
    json.dump(df_num_corrections_clean.to_dict('records'), f, ensure_ascii=False, indent=4)

,language,# edits clean
0,english,15201
1,italian,3982
2,ukrainian,3637
3,german,2380
4,czech,746
5,swedish,408
6,greek,302
7,estonian,90
8,slovene,47
9,latvian,33


In [11]:
df_stats = pd.DataFrame(num_pages)\
             .merge(pd.DataFrame(num_corrections), on=['language'], how='left')\
             .merge(df_num_corrections_clean, on=['language'], how='left')\
             .rename(columns={'# edits': '# edits all', '# edits clean': '# edits'})\
             .sort_values('# edits', ascending=False).set_index('language')
df_stats = df_stats.drop('language_code', axis=1)
df_stats = df_stats.T
# remove later
df_stats['icelandic'] = 0
display(df_stats)

print()
print('|'+'|'+'|'.join(df_stats.columns)+'|')
print('|-|-|-|-|-|-|-|-|-|-|-|-|')
for i, row in df_stats.reset_index().iterrows():
    print('|'+'|'.join([str(i) for i in row.values])+'|')

language,english,italian,ukrainian,german,czech,swedish,greek,estonian,slovene,latvian,icelandic
# pages,9842,2569,1661,1706,479,255,155,48,28,20,0
# edits all,29454,6488,6165,4672,1193,654,562,138,121,75,0
# edits,15201,3982,3637,2380,746,408,302,90,47,33,0



||english|italian|ukrainian|german|czech|swedish|greek|estonian|slovene|latvian|icelandic|
|-|-|-|-|-|-|-|-|-|-|-|-|
|# pages|9842|2569|1661|1706|479|255|155|48|28|20|0|
|# edits all|29454|6488|6165|4672|1193|654|562|138|121|75|0|
|# edits|15201|3982|3637|2380|746|408|302|90|47|33|0|


## Dataset WikiEdits-MultiGEC

In [12]:
columns_dataset_intermidiate = [
    'index', 'text_del', 'text_ins', 'text_del_tag', 'text_ins_tag', 
    'delitions', 'insertions', 'language', 'diff_url'
]
columns_dataset = ['index', 'language', 'text_del_clean', 'text_ins_clean']

df_wiki_metadata = df_output[columns_dataset_intermidiate].rename(columns={'delitions': 'deletions', 'diff_url': 'url'})
display(df_wiki_metadata.tail(3))

df_wiki = df_output[columns_dataset].rename(columns={'text_del_clean': 'text', 'text_ins_clean': 'correction'})
display(df_wiki.tail(3))

,index,text_del,text_ins,text_del_tag,text_ins_tag,deletions,insertions,language,url
26823,26823,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,[['бів']],[['був']],ukrainian,https://uk.wikipedia.org/w/index.php?title=%D0...
26824,26824,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,[['НКВД']],[['НКВС']],ukrainian,https://uk.wikipedia.org/w/index.php?title=%D0...
26825,26825,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,[['зібра но']],[['зібрано']],ukrainian,https://uk.wikipedia.org/w/index.php?title=%D0...


,index,language,text,correction
26823,26823,ukrainian,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...
26824,26824,ukrainian,У жовтні 1939 року о. Йосиф дозволив усім бель...,У жовтні 1939 року о. Йосиф дозволив усім бель...
26825,26825,ukrainian,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...


In [13]:
# save data into WikiEdits-MultiGEC Dataset
df_wiki_metadata.to_parquet('../datasets/WikiEdits-MultiGEC/wikiedits_multi_gec_metadata.parquet', index=False)
df_wiki.to_parquet('../datasets/WikiEdits-MultiGEC/wikiedits_multi_gec.parquet', index=False)

In [14]:
columns_dataset_kaggle = [
    'index', 'text_del', 'text_ins', 'text_del_tag', 'text_ins_tag', 
    'delitions', 'insertions', 'text_del_clean', 'text_ins_clean', 'language', 'diff_url'
]

df_wiki_kaggle = df_output[columns_dataset_kaggle].rename(columns={'text_del_clean': 'text', 'text_ins_clean': 'correction'})
display(df_wiki_kaggle.tail(3))

,index,text_del,text_ins,text_del_tag,text_ins_tag,delitions,insertions,text,correction,language,diff_url
26823,26823,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,[['бів']],[['був']],В післявоєнні роки висока популярність комітет...,В післявоєнні роки висока популярність комітет...,ukrainian,https://uk.wikipedia.org/w/index.php?title=%D0...
26824,26824,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,У жовтні 1939&nbsp;року о. Йосиф дозволив усім...,[['НКВД']],[['НКВС']],У жовтні 1939 року о. Йосиф дозволив усім бель...,У жовтні 1939 року о. Йосиф дозволив усім бель...,ukrainian,https://uk.wikipedia.org/w/index.php?title=%D0...
26825,26825,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,[['зібра но']],[['зібрано']],Засновано село Сілець було у 1465 році удільни...,Засновано село Сілець було у 1465 році удільни...,ukrainian,https://uk.wikipedia.org/w/index.php?title=%D0...


In [16]:
# save data into another formats
df_wiki_kaggle.to_csv('../datasets/WikiEdits-MultiGEC-Kaggle/wikiedits_multi_gec.csv', index=False)
# df_wiki_kaggle.to_pickle('../datasets/WikiEdits-MultiGEC-Kaggle/wikiedits_multi_gec.pkl')

## Viz Language Distribution

In [19]:
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px

In [17]:
df_num_corrections_clean

,language,# edits clean
0,english,15201
1,italian,3982
2,ukrainian,3637
3,german,2380
4,czech,746
5,swedish,408
6,greek,302
7,estonian,90
8,slovene,47
9,latvian,33


In [69]:
fig = px.bar(
    df_num_corrections_clean.sort_values('# edits clean', ascending=True), 
    x="# edits clean", 
    y="language", 
    text_auto='.2s',
)

fig.update_layout(
    showlegend=False,
    xaxis_tickangle=0,
    plot_bgcolor='#FFFFFF',
    autosize=False,
    width=1000,
    height=600,
    
    xaxis=dict(
        title='# edits',
        ticks='outside',
        mirror=False,
        linecolor='black',
        showline=True,
        tickfont=dict(size=16, color='#000'),
        titlefont=dict(size=18, color='#000')
    ),
    yaxis=dict(
        title='Language', 
        ticks='outside',
        mirror=False, 
        linecolor='black',
        showline=True,
        tickfont=dict(size=16, color='#000'),
        titlefont=dict(size=18, color='#000')
    ),
    title_font_family="Times New Roman",
    margin = {'l':25, 'r':25, 't':25, 'b':25},
#     yaxis_range=[0, 10.5],
)
fig.update_traces(
    textfont=dict(size=16), 
    textposition='outside',
    marker_color='#00B894'
)

fig.write_image("images/fig_languages.png", scale=6)
# fig.show()